# Makine Öğrenimi ile Fiyat Konut Tahmini: Feature Engineering & Hibrid Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [2]:
df = pd.read_excel("C:\\konut\\istanbul_konut2.xlsx")

In [3]:
# Eksik değerlerin toplamını ve oranını göster
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

# Eksik değer tablosu oluştur
missing_data = pd.DataFrame({'Eksik Değerler': missing_values, 'Eksik Oranı (%)': missing_percent})
print(missing_data)


               Eksik Değerler  Eksik Oranı (%)
Fiyat                       0         0.000000
İlce                        0         0.000000
Mahalle                     0         0.000000
Metre Kare                  0         0.000000
Oda Sayısı                  2         0.006988
Yaş                        24         0.083852
Bulunduğu Kat             936         3.270212


In [4]:
# 'Yaş' sütunundaki eksik değerleri medyan ile doldur
df['Yaş'] = df['Yaş'].fillna(df['Yaş'].median())
# 'Bulunduğu Kat' sütunundaki eksik değerleri mod ile doldur
df['Bulunduğu Kat'] = df['Bulunduğu Kat'].fillna(df['Bulunduğu Kat'].mode()[0])
# Oda Sayısı sütunundaki eksik değerleri mod (en sık görülen değer) ile doldur
most_frequent_value = df['Oda Sayısı'].mode()[0]
df['Oda Sayısı'] = df['Oda Sayısı'].fillna(most_frequent_value)
print(df.isnull().sum())

Fiyat            0
İlce             0
Mahalle          0
Metre Kare       0
Oda Sayısı       0
Yaş              0
Bulunduğu Kat    0
dtype: int64


In [5]:
df

,Fiyat,İlce,Mahalle,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat
0,5000000,Adalar,Burgazada Mah,55,2.0,24.0,0.0
1,7500000,Adalar,Nizam Mah,65,2.0,31.0,0.0
2,4750000,Adalar,Nizam Mah,60,2.0,34.0,0.0
3,7000000,Adalar,MadeMah,75,2.0,35.0,0.0
4,4200000,Adalar,Nizam Mah,51,2.0,38.0,0.0
...,...,...,...,...,...,...,...
28617,4150000,Zeytinburnu,VeliefendMah.,150,6.0,29.0,1.0
28618,4500000,Zeytinburnu,VeliefendMah.,170,6.0,30.0,1.0
28619,17000000,Zeytinburnu,Gökalp Mah.,350,12.0,16.0,0.0
28620,19000000,Zeytinburnu,Beştelsiz Mah.,500,12.0,28.0,0.0


In [6]:
df

,Fiyat,İlce,Mahalle,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat
0,5000000,Adalar,Burgazada Mah,55,2.0,24.0,0.0
1,7500000,Adalar,Nizam Mah,65,2.0,31.0,0.0
2,4750000,Adalar,Nizam Mah,60,2.0,34.0,0.0
3,7000000,Adalar,MadeMah,75,2.0,35.0,0.0
4,4200000,Adalar,Nizam Mah,51,2.0,38.0,0.0
...,...,...,...,...,...,...,...
28617,4150000,Zeytinburnu,VeliefendMah.,150,6.0,29.0,1.0
28618,4500000,Zeytinburnu,VeliefendMah.,170,6.0,30.0,1.0
28619,17000000,Zeytinburnu,Gökalp Mah.,350,12.0,16.0,0.0
28620,19000000,Zeytinburnu,Beştelsiz Mah.,500,12.0,28.0,0.0


### Target Encoding (Mean Encoding)
Her bir ilçe/mahalle, o ilçe/mahalledeki ortalama konut fiyatı ile değiştirilir

In [7]:
# Target Encoding uygulama
def target_encode(df, target_col):
    # İlce için ortalama fiyatları hesapla
    ilce_means = df.groupby('İlce')[target_col].mean()
    df['Ilce_encoded'] = df['İlce'].map(ilce_means)

    # Mahalle için ortalama fiyatları hesapla
    mahalle_means = df.groupby('Mahalle')[target_col].mean()
    df['Mahalle_encoded'] = df['Mahalle'].map(mahalle_means)

    return df

# Target Encoding'i uygula
df_encoded = target_encode(df.copy(), 'Fiyat')

# Özellikler (X) ve hedef değişkeni (y) belirleyelim
X = df_encoded[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y = df_encoded['Fiyat']

# Veriyi eğitim ve test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest modelini oluştur ve eğit
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# R² skorunu hesapla
r2 = r2_score(y_test, y_pred)

# Sonucu yazdır
print("Target Encoding uygulanmış veri seti ile R² değeri (Random Forest):")
print("R² Score:", r2)


Target Encoding uygulanmış veri seti ile R² değeri (Random Forest):
R² Score: 0.8296936501480221


# Outlier Değerleri Temizleme

## Hibrid Yöntem - (IQR  + Manuel Veri Temizleme)

In [8]:
df_new=df_encoded.copy()

In [9]:
df_new=df_encoded[df_encoded["Oda Sayısı"]<8]
df_new=df_encoded[df_encoded["Yaş"]<57]
df_new = df_encoded[(df_encoded["Metre Kare"] < 270) & (df_encoded["Metre Kare"] > 40)]

In [10]:
X_clean = df_new[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y_clean = df_new['Fiyat']

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Modeli eğitme
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Modelin performansını değerlendirme
y_pred = rf_model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f'Manuel Temizleme Sonrası R^2 Skoru: {r2:.4f}')
print(f'Manuel Temizleme Sonrası Veri sayısı: {len(X_clean)}')

Manuel Temizleme Sonrası R^2 Skoru: 0.8334
Manuel Temizleme Sonrası Veri sayısı: 27961


In [11]:
df_clean = pd.concat([X_clean, y_clean], axis=1)
# İlçe bazında Fiyat, Metre Kare, Oda Sayısı ve Yaş değişkenleri için ortalama ve standart sapma hesaplama
stats = df_clean.groupby('Ilce_encoded').agg(
    Fiyat_mean=('Fiyat', 'mean'), Fiyat_std=('Fiyat', 'std'),
    MetreKare_mean=('Metre Kare', 'mean'), MetreKare_std=('Metre Kare', 'std'),
    OdaSayisi_mean=('Oda Sayısı', 'mean'), OdaSayisi_std=('Oda Sayısı', 'std'),
    Yas_mean=('Yaş', 'mean'), Yas_std=('Yaş', 'std')
).reset_index()

# Aykırı değer eşiğini hesaplama ve kontrol etme
def is_outlier(row, stats):
    ilce_stats = stats[stats['Ilce_encoded'] == row['Ilce_encoded']].iloc[0]
    
    # Eşik değerler
    fiyat_threshold = ilce_stats['Fiyat_mean'] + 2 * ilce_stats['Fiyat_std'] + 1
    metrekare_threshold = ilce_stats['MetreKare_mean'] + 2 * ilce_stats['MetreKare_std'] + 1
    odasayisi_threshold = ilce_stats['OdaSayisi_mean'] + 2 * ilce_stats['OdaSayisi_std'] + 1
    yas_threshold = ilce_stats['Yas_mean'] + 2 * ilce_stats['Yas_std'] + 1

    # Aykırı değer kontrolü
    if (row['Fiyat'] > fiyat_threshold or 
        row['Metre Kare'] > metrekare_threshold or 
        row['Oda Sayısı'] > odasayisi_threshold or 
        row['Yaş'] > yas_threshold):
        return True
    return False

# Aykırı değerleri filtreleme
filtered_data = df_clean[~df_clean.apply(lambda row: is_outlier(row, stats), axis=1)]

# Sonuç
print(f"Orijinal veri boyutu: {len(df_clean)}")
print(f"İlçe Bazlı Temizlenme sonucu  veri boyutu: {len(filtered_data)}")


Orijinal veri boyutu: 27961
İlçe Bazlı Temizlenme sonucu  veri boyutu: 24973


In [12]:
filtered_data

,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat,Ilce_encoded,Mahalle_encoded,Fiyat
0,55,2.0,24.0,0.0,1.047431e+07,1.064654e+07,5000000
1,65,2.0,31.0,0.0,1.047431e+07,1.104514e+07,7500000
2,60,2.0,34.0,0.0,1.047431e+07,1.104514e+07,4750000
3,75,2.0,35.0,0.0,1.047431e+07,9.944185e+06,7000000
4,51,2.0,38.0,0.0,1.047431e+07,1.104514e+07,4200000
...,...,...,...,...,...,...,...
28604,150,5.0,30.0,0.0,4.050061e+06,1.965306e+06,3000000
28607,125,5.0,31.0,0.0,4.050061e+06,4.250833e+06,2650000
28608,150,5.0,31.0,1.0,4.050061e+06,3.729308e+06,4449000
28614,150,6.0,25.0,1.0,4.050061e+06,3.708659e+06,4200000


In [13]:
# df_clean = pd.concat([X_clean, y_clean], axis=1)

In [14]:
# Özellikler ve hedef değişkeni belirleme
X = filtered_data[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y = filtered_data['Fiyat']

# IQR yöntemiyle aykırı değerleri temizleme
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 2 * IQR
upper_bound = Q3 + 2 * IQR
mask = ~((X < lower_bound) | (X > upper_bound)).any(axis=1)
X_clean = X[mask]
y_clean = y[mask]

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Modeli eğitme
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Modelin performansını değerlendirme
y_pred = rf_model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f'R^2 Skoru: {r2:.4f}')
print(f'IQR sonrası Veri boyutu: {len(X_clean)}')

R^2 Skoru: 0.8932
IQR sonrası Veri boyutu: 24129
